A file to test forward hook methods, to extract input for each layer
https://kozodoi.me/python/deep%20learning/pytorch/tutorial/2021/05/27/extracting-features.html#2.-Why-do-we-need-intermediate-features?

In [ ]:
%%capture
import torch
from transformers import AutoTokenizer, OPTForCausalLM, pipeline
from datasets import load_dataset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = load_dataset('c4', 'en', streaming=True)
model_text = "facebook/opt-125m"
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_text)

# Load model with pre-trained head
model = OPTForCausalLM.from_pretrained(model_text, output_attentions=True, output_hidden_states=True)

generator = pipeline('text-generation', model=model_text)

calibration_data = []
for i, data in enumerate(iter(dataset['train'])):
    if i > 10:
        break
    tokenized = tokenizer.encode(data['text'], return_tensors="pt", padding="max_length", truncation=True, max_length=512)
    calibration_data.append(tokenized)
calibration_data = torch.squeeze(torch.stack(calibration_data)).to(device=device)

In [ ]:
# Load model with pre-trained head
model = OPTForCausalLM.from_pretrained(model_text, output_attentions=True, output_hidden_states=True)

# Store features from batch
features = {}

def get_features(name):
    def pre_hook(model, input):
        # print(input)
        try:
            print(f"for {name}, input size is {input[0].shape}")
        except:
            pass
        features[name] = input
    return pre_hook


# features_2 = {}
# def global_pre_hook():
#     def pre_hook(model, input):
#         features_2[model.name] = input
#     return pre_hook

# model.register_module_forward_pre_hook(global_pre_hook())
for n, m in model.named_modules():
    new_hook = get_features(n)
    # print(m)
    m.register_forward_pre_hook(new_hook)
# model.register_forward_hook(get_features('feats'))


In [ ]:

preds = model(calibration_data)
# print(features)

In [ ]:
for k in features.keys():
    # print(k)
    try:
        print(features[k])
    except:
        continue